In [ ]:
import pandas as pd
import numpy as np
import os, sys, gc

In [ ]:
path = '/home/workspace/data/.train/.task152/train/'

In [ ]:
brand = pd.read_csv(path + '6_brand_sales.csv')
weather = pd.read_csv(path + '7_weather.csv')

brand['sales'] = brand['sales'].apply(lambda x: x.replace('-', '0'))
brand['sales'] = brand['sales'].astype(int)

In [ ]:
# 시드를 고정하는 함수
import random
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
seed_everything(0)

In [ ]:
train = pd.read_csv(path + '1_daily_sales.csv')
stock = pd.read_csv(path + '2_stock.csv') # 2016년도부터 정보가 있음 
gdp = pd.read_csv(path + '4_gdp.csv')  # 분기로 값이 나와있어서 월로 변경 필요 
car_registration = pd.read_csv(path + '5_car_registration.csv') 
exchange_rate = pd.read_csv(path + '8_exchange_rate.csv') 

In [ ]:
train = train.groupby('YYYYMMDD')['Qty'].agg({'sum'}).reset_index()

In [ ]:
# !pip install workalendar
from workalendar.asia import SouthKorea # 한국의 공휴일, version : 1.1.1
import datetime 

train['Time'] = pd.to_datetime(train['YYYYMMDD'].astype(str).apply(lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:]))

'''
from datetime import timedelta, datetime
datelist = pd.date_range('2015-01-01', periods=365*5-91).tolist() # pd.date_range(start="2015-01-01",end="2020-12-31")

datelist_ = []
for i in datelist:
    datelist_.append(i.strftime("%Y-%m-%d"))
datedf = pd.DataFrame(datelist_)
datedf.columns = ['Time']
datedf['Time'] = pd.to_datetime(datedf['Time'])

datedf = datedf.merge(train, how='left', on='Time')
# datedf['sum'] = datedf['sum'].fillna(0)
datedf['YYYYMMDD'] = 10000 * datedf['Time'].dt.year + 100 * datedf['Time'].dt.month + 1 * datedf['Time'].dt.day
train = datedf.copy()
'''

train['year'] = train['Time'].dt.year
train['month'] = train['Time'].dt.month
train['quarter'] = train['Time'].dt.quarter
train['week'] = train['Time'].dt.week
train['weekday'] = train['Time'].dt.weekday
train['day'] = train['Time'].dt.day
train['Date'] = train['year'].astype(str) + 'Q' + train['quarter'].astype(str)
train['YYMM'] = train['YYYYMMDD'].astype(str).apply(lambda x: x[2:6])

# holidays = pd.concat([pd.Series(np.array(SouthKorea().holidays(2019))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2018))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2017))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2016))[:, 0]), pd.Series(np.array(SouthKorea().holidays(2015))[:, 0])]).reset_index(drop=True)
# train['holiday'] = train['Time'].dt.date.isin(holidays).astype(int)
train['weekend'] = train['weekday'].map({0:0, 1:0, 2:0, 3:0, 4:0, 5:1, 6:1})
# train['is_holiday'] = (train['weekend'] + train['holiday']).map({0:0, 1:1, 2:1})
train['date'] = pd.to_datetime(train['Time'].dt.date)

In [ ]:
neujahr = pd.DataFrame({
  'holiday': 'neujahr',
  'ds': pd.to_datetime(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01'])
})

karfreitag = pd.DataFrame({
  'holiday': 'karfreitag',
  'ds': pd.to_datetime(['2015-04-03', '2015-04-06', '2016-03-25', '2016-03-28', '2017-04-14', '2017-04-17', 
                       '2018-03-30', '2018-04-02', '2019-04-19', '2019-04-22'])
})

tagderarbeit = pd.DataFrame({
  'holiday': 'tagderarbeit',
  'ds': pd.to_datetime(['2015-05-01', '2016-05-01', '2017-05-01', '2018-05-01', '2019-05-01'])
})

ChristiHimmelfahrt = pd.DataFrame({
  'holiday': 'ChristiHimmelfahrt',
  'ds': pd.to_datetime(['2015-05-14', '2016-05-05', '2017-05-25', '2018-05-10', '2019-05-30'])
})

pfingetmontag = pd.DataFrame({
  'holiday': 'pfingetmontag',
  'ds': pd.to_datetime(['2015-05-25', '2016-05-16', '2017-06-05', '2018-05-21', '2019-06-10'])
})

tagderdeuscheneinheit = pd.DataFrame({
  'holiday': 'tagderdeuscheneinheit',
  'ds': pd.to_datetime(['2015-10-03', '2016-10-03', '2017-10-03', '2018-10-03', '2019-10-03'])
})

chrismas = pd.DataFrame({
  'holiday': 'chrismas',
  'ds': pd.to_datetime(['2015-12-25', '2016-12-25', '2017-12-25', '2018-12-25', '2019-12-25', 
                       '2015-12-26', '2016-12-26', '2017-12-26', '2018-12-26', '2019-12-26'])
})

holidays = pd.concat([neujahr, karfreitag, tagderarbeit, ChristiHimmelfahrt, pfingetmontag, 
                      tagderdeuscheneinheit, chrismas
                     ], axis=0)

train['holiday'] = train['Time'].dt.date.isin(holidays).astype(int)

In [ ]:
weather = weather.groupby(['yymm']).agg({'mean'}).reset_index()

del weather['region']
weather.columns = ['YYMM', 'MAX_TEMP', 'MIN_TEMP', 'AVG_TEMP', 'SNOW_FALL_DAYS', 'SNOW_FALL_CM']

stock = stock[stock['YYMM'] > 1512].reset_index(drop=True)
stock = stock.astype(np.int)

stock['YYMM'] = stock['YYMM'].astype(str)
stock.columns =['YYMM', 'WT', 'ASWT', 'ST']
train = train.merge(stock, how='left', on='YYMM')

gdp.columns = ['Date', 'QGDP', 'QGDPG', 'QGDPAG']
train = train.merge(gdp, how='left', on='Date')

car_registration.columns = ['YYMM', 'cvm', 'pvm']
car_registration['YYMM'] = car_registration['YYMM'].astype(str)
train = train.merge(car_registration, how='left', on='YYMM')

exchange_rate.columns = ['YYYYMMDD', 'meme']
train = train.merge(exchange_rate, how='left', on='YYYYMMDD')

weather['YYMM'] = weather['YYMM'].astype(str)
train = train.merge(weather, how='left', on='YYMM')

In [ ]:
n_splits= 5
seed = 8343
n_seeds = 1

NUM_BOOST_ROUND = 2000
lgbm_param = {'objective':'Regression',
              'metric':'RMSE',
              'boosting_type': 'gbdt',
              'random_state':seed,
              'learning_rate':0.01,
              'subsample':0.8,
              'tree_learner': 'serial',
              'max_depth' : 16, 
#               'reg_lambda':5,
#               'reg_alpha': 7,
                'num_leaves': 127,   
              'verbosity' : -1
            }

In [ ]:
import lightgbm as lgbm
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import TimeSeriesSplit
from tqdm.notebook import tqdm
from sklearn.metrics import roc_auc_score, mean_squared_error
import pickle 

preds = []
valid_preds = []
feature_importances_full = pd.DataFrame()
for te in tqdm(range(1, 32)):
    traindata = train.copy()
    
    #######################
    ### 공통 변수 생성 ###
    traindata['YYMM'] = traindata['YYMM'].astype(int)
    for i in range(1, 2):
        traindata['next_yyyymm'] = traindata['YYMM'] + 100
        temp_dict = traindata.groupby('next_yyyymm')['sum'].mean()
        traindata['before_yyyymm_{}'.format(i)] = traindata['YYYYMMDD'].map(temp_dict)    ### 작년 같은 주, 달의 평균 
    
    ### 최근 12개월의 평균 ### 
    for i in range(1, 13):
        traindata['next_month'] = traindata['month'] + i
        temp_dict = traindata.groupby('next_month')['sum'].mean()
        traindata['before_month_{}'.format(i)] = traindata['week'].map(temp_dict)
    
    #######################
    ### 개별 변수 생성 ###
    ### 최근 30일간의 거래 ### 
    for i in range(te, 31):
        traindata['sum_shift{}'.format(i)] = traindata['sum'].shift(i)
        
    ### 최근 8주간의 거래 ###
    for i in range((te//5)+1, 9):
        traindata['next_week'] = traindata['week'] + i
        temp_dict = traindata.groupby('next_week')['sum'].mean()
        traindata['before_week_{}'.format(i)] = traindata['week'].map(temp_dict)    
        
    # Target 을 Shift 해주는 작업 진행 
    features = [c for c in traindata.columns if c not in ['', 'next_month', 'next_week', 'date', 'YYYYMMDD', 'sum', 'Time', 'hour', 'YYMM', 'Date', 'tomorrow']]

    traindata['target'] = traindata['sum'].shift(-te)
    traindata = traindata[traindata['target'].notnull()].reset_index(drop=True)
    traindata = traindata[traindata['year'] > 2015].reset_index(drop=True)
    
    # 변수중요도 기록했다가 하위 n개 삭제해야함 
    testdata = traindata.loc[traindata.shape[0] - te]
    X_test = pd.DataFrame(testdata[features]).T.reset_index(drop=True).astype(float)
                                                   
    traindata = traindata.loc[0:traindata.shape[0] - te - 1]
    
    X_train = traindata[features].astype(float)
    y_train = traindata['target']    
    
    # Train Set / Valid Set 생성 
    test_pred = 0
    valid_pred = np.zeros(len(X_train))
    feature_importances = pd.DataFrame()
    feature_importances['feature'] = features
    
    # Group Kfold Year로 변경 
    # kfolds = KFold(n_splits=5, random_state=0, shuffle=True)
    kfolds = GroupKFold(n_splits=4)
    for n_fold, (trn_idx, val_idx) in enumerate(kfolds.split(X_train, y_train, groups=X_train['year'])):
        
        trn_x, trn_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx]
        val_x, val_y = X_train.iloc[val_idx], y_train.iloc[val_idx]

        dtrain = lgbm.Dataset( trn_x, trn_y )
        dvalid = lgbm.Dataset( val_x, val_y ,reference=dtrain)

        model = lgbm.train(lgbm_param , dtrain, NUM_BOOST_ROUND, valid_sets=(dtrain, dvalid), valid_names=('train','valid'), 
                            verbose_eval = False, early_stopping_rounds=50)
        
        with open('lightgbm_10month_{}day_{}fold.pkl'.format(te, n_fold), 'wb') as f:
            pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)    
        
        valid_pred[val_idx] = model.predict(X_train.iloc[val_idx])
        test_pred  += model.predict(X_test) / 5
        
                
    feature_importances['importance'] = model.feature_importance()
    valid_preds.append(mean_squared_error(y_train, valid_pred) ** 0.5)
    feature_importances_full = pd.concat([feature_importances_full, feature_importances], axis=0)
    preds.append(test_pred)
print(np.mean(valid_preds), np.median(valid_preds), np.std(valid_preds))

In [ ]:
fe_imp = feature_importances_full.groupby(['feature'])['importance'].agg({'mean'}).reset_index()
fe_imp.sort_values(by='mean', ascending=False).head(20)

In [ ]:
from datetime import timedelta, datetime
datelist = pd.date_range(start="2019-10-01",end="2019-10-31")

datelist_ = []
for i in datelist:
    datelist_.append(i.strftime("%Y-%m-%d"))
datedf = pd.DataFrame(datelist_)
datedf.columns = ['Time']
datedf['Time'] = pd.to_datetime(datedf['Time'])

datedf['YYYYMMDD'] = 10000 * datedf['Time'].dt.year + 100 * datedf['Time'].dt.month + 1 * datedf['Time'].dt.day
del datedf['Time']

datedf['Qty'] = preds[0:31]
datedf['Qty'] = datedf['Qty'].apply(lambda x: x[0])

sub = pd.read_csv('/home/workspace/data/baseline/task152/prediction/prediction.csv')
sub['Time'] = pd.to_datetime(sub['YYYYMMDD'].astype(str).apply(lambda x: x[0:4] + '-' + x[4:6] + '-' + x[6:]))

submission = pd.merge(sub, datedf, how='left', on ='YYYYMMDD')
submission = submission[submission['Qty_y'].notnull()].reset_index(drop=True)
submission = submission[['YYYYMMDD', 'Qty_y']].rename(columns={'Qty_y':'Qty'})
submission.to_csv("/home/workspace/user-workspace/prediction/prediction(LGB).csv", index=False)

In [ ]:
import os
# prediction.to_csv("/home/workspace/user-workspace/prediction/prediction(Prophet1).csv", index=False)
from nipa.taskSubmit import nipa_submit

team_id = "1192"
task_no= "152"

prediction_path = "/home/workspace/user-workspace/prediction/prediction(LGB).csv"
# 파일 존재 여부 확인
print("is file: ", os.path.isfile(prediction_path))

# 제출 성공
nipa_submit(team_id=team_id,
task_no=task_no,
result=prediction_path
)